  ###  `ref` ,  `val` ,  `uval`  Modifiers for  `linear`  Clause

 When generating vector functions from  `declare` ~ `simd`  directives, it is important for a compiler to know the proper types of function arguments in order to generate efficient s. This is especially true for C++ reference types and Fortran arguments.

 In the following example, the function  _add_one2_  has a C++ reference parameter (or Fortran argument)  _p_ .  Variable  _p_  gets incremented by 1 in the function. The caller loop  _i_  in the main program passes  a variable  _k_  as a reference to the function  _add_one2_  call.    The  `ref`  modifier for the  `linear`  clause on the   `declare` ~ `simd`  directive is used to annotate the  reference-type parameter  _p_  to match the property of the variable   _k_  in the loop.   This use of reference type is equivalent to the second call to   _add_one2_  with a direct passing of the array element  _a[i]_ .   In the example, the preferred vector  length 8 is specified for both the caller loop and the callee function.

 When  `linear(ref(p))`  is applied to an argument passed by reference,  it tells the compiler that the addresses in its vector argument are consecutive, and so the compiler can generate a single vector load or store instead of  a gather or scatter. This allows more efficient SIMD  to be generated with  less source changes.

In [ ]:

/*
* name: linear_modifier.1cpp
* type: C++
* version: omp_4.5
*/
#include <stdio.h>

#define NN 1023
int a[NN];

#pragma omp declare simd linear(ref(p)) simdlen(8)
void add_one2(int& p)
{
   p += 1;
}

int main(void)
{
   int i;
   int* p = a;

   for (i = 0; i < NN; i++) {
      a[i] = i;
   }

#pragma omp simd linear(p) simdlen(8)
   for (i = 0; i < NN; i++) {
      int& k = *p;
      add_one2(k);
      add_one2(a[i]);
      p++;
   }

   for (i = 0; i < NN; i++) {
      if (a[i] != i+2) {
         printf("failed\n");
         return 1;
      }
   }
   printf("passed\n");
   return 0;
}



In [ ]:

! name: linear_modifier.1.f90
! type: F-free
! version: omp_4.5
module m
   integer, parameter :: NN = 1023
   integer :: a(NN)

 contains
   subroutine add_one2(p)
   !$omp declare simd(add_one2) linear(ref(p)) simdlen(8)
   implicit none
   integer :: p

   p = p + 1
   end subroutine
end module

program main
   use m
   implicit none
   integer :: i, p

   do i = 1, NN
      a(i) = i
   end do

   p = 1
   !$omp simd linear(p) simdlen(8)
   do i = 1, NN
      associate(k => a(p))
         call add_one2(k)
      end associate
      call add_one2(a(i))
      p = p + 1
   end do

   do i = 1, NN
      if (a(i) /= i+2) then
         print *, "failed"
         stop
      endif
   end do
   print *, "passed"
end program




   The following example is a variant of the above example. The function  _add_one2_  in the C++  includes an additional C++ reference parameter  _i_ .  The loop index  _i_  of the caller loop  _i_  in the main program  is passed as a reference to the function  _add_one2_  call.    The loop index  _i_  has a uniform address with linear value of step 1 across SIMD lanes.  Thus, the  `uval`  modifier is used for the  `linear`  clause  to annotate the C++ reference-type parameter  _i_  to match  the property of loop index  _i_ .

 In the correponding Fortran  the arguments  _p_  and  _i_  in the routine  _add_on2_  are passed by references.  Similar modifiers are used for these variables in the  `linear`  clauses to match with the property at the caller loop in the main program.

 When  `linear(uval(i))`  is applied to an argument passed by reference, it  tells the compiler that its addresses in the vector argument are uniform  so that the compiler can generate a scalar load or scalar store and create  linear values. This allows more efficient SIMD  to be generated with  less source changes.

In [ ]:

/*
* name: linear_modifier.2cpp
* type: C++
* version: omp_4.5
*/
#include <stdio.h>

#define NN 1023
int a[NN];

#pragma omp declare simd linear(ref(p)) linear(uval(i))
void add_one2(int& p, const int& i)
{
   p += i;
}

int main(void)
{
   int i;
   int* p = a;

   for (i = 0; i < NN; i++) {
      a[i] = i;
   }

   #pragma omp simd linear(p)
   for (i = 0; i < NN; i++) {
      int& k = *p;
      add_one2(k, i);
      p++;
   }

   for (i = 0; i < NN; i++) {
      if (a[i] != i*2) {
         printf("failed\n");
         return 1;
      }
   }
   printf("passed\n");
   return 0;
}



In [ ]:

! name: linear_modifier.2f90
! type: F-free
! version: omp_4.5
module m
   integer, parameter :: NN = 1023
   integer :: a(NN)

 contains
   subroutine add_one2(p, i)
   !$omp declare simd(add_one2) linear(ref(p)) linear(uval(i))
   implicit none
   integer :: p
   integer, intent(in) :: i

   p = p + i
   end subroutine
end module

program main
   use m
   implicit none
   integer :: i, p

   do i = 1, NN
      a(i) = i
   end do

   p = 1
   !$omp simd linear(p)
   do i = 1, NN
      call add_one2(a(p), i)
      p = p + 1
   end do

   do i = 1, NN
      if (a(i) /= i*2) then
         print *, "failed"
         stop
      endif
   end do
   print *, "passed"
end program



 In the following example, the function  _func_  takes arrays  _x_  and  _y_  as arguments, and accesses the array elements referenced by  the index  _i_ . The caller loop  _i_  in the main program passes a linear copy of the variable  _k_  to the function  _func_ . The  `val`  modifier is used for the  `linear`  clause  in the  `declare` ~ `simd`  directive for the function  _func_  to annotate argument  _i_  to match the property of  the actual argument  _k_  passed in the SIMD loop. Arrays  _x_  and  _y_  have uniform addresses across SIMD lanes.

 When  `linear(val(i):1)`  is applied to an argument,  it tells the compiler that its addresses in the vector argument may not be  consecutive, however, their values are linear (with stride 1 here). When the value of  _i_  is used  in subscript of array references (e.g.,  _x[i]_ ), the compiler can generate  a vector load or store instead of a gather or scatter. This allows more  efficient SIMD  to be generated with less source changes.

In [ ]:

/*
* name: linear_modifier.3c
* type: C
* version: omp_4.5
*/
#include <stdio.h>

#define N 128

#pragma omp declare simd simdlen(4) uniform(x, y) linear(val(i):1)
double func(double x[], double y[], int i)
{
   return (x[i] + y[i]);
}

int main(void)
{
   double x[N], y[N], z1[N], z2;
   int i, k;

   for (i = 0; i < N; i++) {
      x[i] = (double)i;
      y[i] = (double)i*2;
   }

   k = 0;
#pragma omp simd linear(k)
   for (i = 0; i < N; i++) {
      z1[i] = func(x, y, k);
      k++;
   }

   for (i = 0; i < N; i++) {
      z2 = (double)(i + i*2);
      if (z1[i] != z2) {
         printf("failed\n");
         return 1;
      }
   }
   printf("passed\n");
   return 0;
}



In [ ]:

! name: linear_modifier.3f
! type: F-free
! version: omp_4.5
module func_mod
contains
   real(8) function func(x, y, i)
!$omp declare simd(func) simdlen(4) uniform(x, y) linear(val(i):1)
      implicit none
      real(8), intent(in) :: x(*), y(*)
      integer, intent(in) :: i

      func = x(i) + y(i)

   end function func
end module func_mod

program main
   use func_mod
   implicit none
   integer, parameter :: n = 128
   real(8) :: x(n), y(n), z1(n), z2
   integer :: i, k

   do i=1, n
      x(i) = real(i, kind=8)
      y(i) = real(i*2, kind=8)
   enddo

   k = 1
!$omp simd linear(k)
   do i=1, n
      z1(i) = func(x, y, k)
      k = k + 1
   enddo

   do i=1, n
      z2 = real(i+i*2, kind=8)
      if (z1(i) /= z2) then
         print *, 'failed'
         stop
      endif
   enddo
   print *, 'passed'
end program main

